# Docling PDF Reader — RAG with rich grounding

In [ ]:
from rich.pretty import pprint
import warnings
import os
from dotenv import load_dotenv

load_dotenv()
warnings.filterwarnings(action="ignore", category=FutureWarning, module="easyocr")
warnings.filterwarnings(action="ignore", category=UserWarning, module="pydantic")

## Basic usage

In order to load PDF data with Docling, we use a `DoclingPDFReader`.

To leverage Docling's rich metadata for grounding purposes, we configure:
- `export_type` to JSON, for utilizing Docling's native representation (JSON), and
- `chunk_docs` to True, for having the reader object automatically chunk that JSON representation for us (otherwise we will get LlamaIndex Documents with raw JSON content)

In [ ]:
from llama_index.readers.docling.base import DoclingPDFReader

reader = DoclingPDFReader(
    export_type=DoclingPDFReader.ExportType.JSON,  # rich JSON format or Markdown export
    chunk_docs=True,  # whether to chunk the docs already within the reader or return the raw content
)
nodes = reader.load_data(
    file_path="https://arxiv.org/pdf/2408.09869",  # PDF local path or URL (or iterable thereof)
)

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Let's preview a sample node (chunk):

In [ ]:
pprint(nodes[5], max_length=5, max_string=250, max_depth=2)

TextNode(
│   id_='1a7e20ed-c6b4-484b-98c3-a5f729b91d0d',
│   embedding=None,
│   metadata={
│   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   'path': '#/main-text/21',
│   │   'heading': '2 Getting Started',
│   │   'page': 2,
│   │   'bbox': [...]
│   },
│   excluded_embed_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   excluded_llm_metadata_keys=['dl_doc_hash', 'origin', 'path', 'page', 'bbox', ... +1],
│   relationships={
│   │   <NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(...),
│   │   <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(...)
│   },
│   text='To use Docling, you can simply install the docling package from PyPI. Documentation and examples are available in our GitHub repository at github.com/DS4SD/docling. All required model assets 1 are downloaded to a local huggingface datasets cache on f'+71,
│   mimetype='text/plain',
│   start_char_idx=8531,
│   end_char_idx=8852,
│   text_template='{metadata_str}\n\n{content}',
│   metadata_template='{key}: {value}',
│   metadata_seperator='\n'
)

## RAG demo

Setting up the embed model:

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")

Setting up the vector store:

In [ ]:
from tempfile import TemporaryDirectory
from llama_index.vector_stores.milvus import MilvusVectorStore

vector_store = MilvusVectorStore(
    uri=os.environ.get(
        "MILVUS_URL", default=f"{(tmp_dir := TemporaryDirectory()).name}/milvus_demo.db"
    ),
    collection_name="docling_collection",
    dim=len(embed_model.get_text_embedding("hi")),
    overwrite=True,
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Setting up the index:

In [ ]:
from llama_index.core import StorageContext, VectorStoreIndex

storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex(
    nodes=nodes,
    embed_model=embed_model,
    storage_context=storage_context,
    show_progress=True,
)

Generating embeddings:   0%|          | 0/83 [00:00<?, ?it/s]

Setting up the LLM:

In [ ]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI

HF_TOKEN = os.environ.get("HF_TOKEN")

llm = HuggingFaceInferenceAPI(
    token=HF_TOKEN,
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
)

And now we are ready to ask questions on our document content.

As shown below, besides the response itself, we are getting the PDF-level grounding, incl. page number and bounding box information:

In [ ]:
query_engine = index.as_query_engine(llm=llm)
query_res = query_engine.query("Can I use OCR with Docling?")
pprint(query_res, max_length=5, max_string=70, max_depth=4)

Response(
│   response='\nYes, Docling provides optional support for OCR to cover scanned PDFs '+407,
│   source_nodes=[
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='71c6ea28-32ce-4057-b7eb-1993923c2c1e',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='Docling provides optional support for OCR, for example to cover scanne'+445,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=19404,
│   │   │   │   end_char_idx=19919,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.8330934047698975
│   │   ),
│   │   NodeWithScore(
│   │   │   node=TextNode(
│   │   │   │   id_='ec655d18-add9-4df8-a643-5f5a254688bd',
│   │   │   │   embedding=None,
│   │   │   │   metadata={...},
│   │   │   │   excluded_embed_metadata_keys=[...],
│   │   │   │   excluded_llm_metadata_keys=[...],
│   │   │   │   relationships={...},
│   │   │   │   text='We are actively seeking collaboration from the open-source community t'+69,
│   │   │   │   mimetype='text/plain',
│   │   │   │   start_char_idx=20111,
│   │   │   │   end_char_idx=20250,
│   │   │   │   text_template='{metadata_str}\n\n{content}',
│   │   │   │   metadata_template='{key}: {value}',
│   │   │   │   metadata_seperator='\n'
│   │   │   ),
│   │   │   score=0.8110546469688416
│   │   )
│   ],
│   metadata={
│   │   '71c6ea28-32ce-4057-b7eb-1993923c2c1e': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'path': '#/main-text/45',
│   │   │   'heading': 'OCR',
│   │   │   'page': 4,
│   │   │   'bbox': [107.38465881347656, 568.1728515625, 504.0063171386719, 632.801513671875]
│   │   },
│   │   'ec655d18-add9-4df8-a643-5f5a254688bd': {
│   │   │   'dl_doc_hash': '556ad9e23b6d2245e36b3208758cf0c8a709382bb4c859eacfe8e73b14e635aa',
│   │   │   'path': '#/main-text/46',
│   │   │   'heading': 'OCR',
│   │   │   'page': 4,
│   │   │   'bbox': [107.57046508789062, 540.989990234375, 504.0033874511719, 561.63427734375]
│   │   }
│   }
)